### Summary
The purpose of this notebook is to clean raw Ledning Club data. This is the first part of a project aimed at creating a loan classification model for conservative investors in Lending Club.  The notebook shows the steps taken to prepare the raw Lending Club dataset for exploratory data analysis and machine learning. A brief summary of the content of this notebook is below:

**Removing Extraneous Data**
1. Removing columns with 100% missing values.
2. Removing columns based on description that: 
    * Leaked information from the future.
    * Contained redundant information.
3. Removing columns with only one unique value.  

**Preparing features for data exploration and machine learning**
1. Preparing Categorical columns by:
    * Mapping ordinal values to integers.
    * Encoding nominal values as dummy variables.
2. Removing percentage signs from continous data. 
3. Preparing the target column.
4. Handling missing values by:
    * Dropping rows with missing values under certain criteria.
    * Imputing missing values using observations from data and a consevative mindset.

## Importing the data

In [65]:
# importing relevant packages
import pandas as pd
import math

# importing the dataset
accepted_loans = pd.read_csv('accepted_2007_to_2018Q4.csv.gz', compression='gzip')

/var/folders/6_/pv__40bn4_375n00qt4w9fmcs_vd98/T/ipykernel_55644/489581963.py:6: DtypeWarning: Columns (0,19,49,59,118,129,130,131,134,135,136,139,145,146,147) have mixed types. Specify dtype option on import or set low_memory=False.
  accepted_loans = pd.read_csv('accepted_2007_to_2018Q4.csv.gz', compression='gzip')


In [66]:
# viewing the size of the dataset
print('The size of the dataset: ' + str(accepted_loans.shape))

# viewing the first few columns of the dataset
accepted_loans.head()

The size of the dataset: (2260701, 151)


,id,member_id,loan_amnt,funded_amnt,funded_amnt_inv,term,int_rate,installment,grade,sub_grade,emp_title,emp_length,home_ownership,annual_inc,verification_status,issue_d,loan_status,pymnt_plan,url,desc,purpose,title,zip_code,addr_state,dti,delinq_2yrs,earliest_cr_line,fico_range_low,fico_range_high,inq_last_6mths,mths_since_last_delinq,mths_since_last_record,open_acc,pub_rec,revol_bal,revol_util,total_acc,initial_list_status,out_prncp,out_prncp_inv,total_pymnt,total_pymnt_inv,total_rec_prncp,total_rec_int,total_rec_late_fee,recoveries,collection_recovery_fee,last_pymnt_d,last_pymnt_amnt,next_pymnt_d,last_credit_pull_d,last_fico_range_high,last_fico_range_low,collections_12_mths_ex_med,mths_since_last_major_derog,policy_code,application_type,annual_inc_joint,dti_joint,verification_status_joint,acc_now_delinq,tot_coll_amt,tot_cur_bal,open_acc_6m,open_act_il,open_il_12m,open_il_24m,mths_since_rcnt_il,total_bal_il,il_util,open_rv_12m,open_rv_24m,max_bal_bc,all_util,total_rev_hi_lim,inq_fi,total_cu_tl,inq_last_12m,acc_open_past_24mths,avg_cur_bal,bc_open_to_buy,bc_util,chargeoff_within_12_mths,delinq_amnt,mo_sin_old_il_acct,mo_sin_old_rev_tl_op,mo_sin_rcnt_rev_tl_op,mo_sin_rcnt_tl,mort_acc,mths_since_recent_bc,mths_since_recent_bc_dlq,mths_since_recent_inq,mths_since_recent_revol_delinq,num_accts_ever_120_pd,num_actv_bc_tl,num_actv_rev_tl,num_bc_sats,num_bc_tl,num_il_tl,num_op_rev_tl,num_rev_accts,num_rev_tl_bal_gt_0,num_sats,num_tl_120dpd_2m,num_tl_30dpd,num_tl_90g_dpd_24m,num_tl_op_past_12m,pct_tl_nvr_dlq,percent_bc_gt_75,pub_rec_bankruptcies,tax_liens,tot_hi_cred_lim,total_bal_ex_mort,total_bc_limit,total_il_high_credit_limit,revol_bal_joint,sec_app_fico_range_low,sec_app_fico_range_high,sec_app_earliest_cr_line,sec_app_inq_last_6mths,sec_app_mort_acc,sec_app_open_acc,sec_app_revol_util,sec_app_open_act_il,sec_app_num_rev_accts,sec_app_chargeoff_within_12_mths,sec_app_collections_12_mths_ex_med,sec_app_mths_since_last_major_derog,hardship_flag,hardship_type,hardship_reason,hardship_status,deferral_term,hardship_amount,hardship_start_date,hardship_end_date,payment_plan_start_date,hardship_length,hardship_dpd,hardship_loan_status,orig_projected_additional_accrued_interest,hardship_payoff_balance_amount,hardship_last_payment_amount,disbursement_method,debt_settlement_flag,debt_settlement_flag_date,settlement_status,settlement_date,settlement_amount,settlement_percentage,settlement_term
0,68407277,NaN,3600.0,3600.0,3600.0,36 months,13.99,123.03,C,C4,leadman,10+ years,MORTGAGE,55000.0,Not Verified,Dec-2015,Fully Paid,n,https://lendingclub.com/browse/loanDetail.acti...,NaN,debt_consolidation,Debt consolidation,190xx,PA,5.91,0.0,Aug-2003,675.0,679.0,1.0,30.0,NaN,7.0,0.0,2765.0,29.7,13.0,w,0.00,0.00,4421.723917,4421.72,3600.00,821.72,0.0,0.0,0.0,Jan-2019,122.67,NaN,Mar-2019,564.0,560.0,0.0,30.0,1.0,Individual,NaN,NaN,NaN,0.0,722.0,144904.0,2.0,2.0,0.0,1.0,21.0,4981.0,36.0,3.0,3.0,722.0,34.0,9300.0,3.0,1.0,4.0,4.0,20701.0,1506.0,37.2,0.0,0.0,148.0,128.0,3.0,3.0,1.0,4.0,69.0,4.0,69.0,2.0,2.0,4.0,2.0,5.0,3.0,4.0,9.0,4.0,7.0,0.0,0.0,0.0,3.0,76.9,0.0,0.0,0.0,178050.0,7746.0,2400.0,13734.0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,N,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,Cash,N,NaN,NaN,NaN,NaN,NaN,NaN
1,68355089,NaN,24700.0,24700.0,24700.0,36 months,11.99,820.28,C,C1,Engineer,10+ years,MORTGAGE,65000.0,Not Verified,Dec-2015,Fully Paid,n,https://lendingclub.com/browse/loanDetail.acti...,NaN,small_business,Business,577xx,SD,16.06,1.0,Dec-1999,715.0,719.0,4.0,6.0,NaN,22.0,0.0,21470.0,19.2,38.0,w,0.00,0.00,25679.660000,25679.66,24700.00,979.66,0.0,0.0,0.0,Jun-2016,926.35,NaN,Mar-2019,699.0,695.0,0.0,NaN,1.0,Individual,NaN,NaN,NaN,0.0,0.0,204396.0,1.0,1.0,0.0,1.0,19.0,18005.0,73.0,2.0,3.0,6472.0,29.0,111800.0,0.0,0.0,6.0,4.0,9733.0,57830.0,27.1,0.0,0.0,113.0,192.0,2.0,2.0,4.0,2.0,NaN,0.0,6.0,0.0,5.0,5.0,13.0,17.0,6.0,20.0,27.0,5.0,22.0,0.0,0.0,0.0,2.0,97.4,7.7,0.0,0.0,314017.0,39475.0,79300.0,24667.0,NaN,NaN,NaN,NaN,NaN,N

From the above output we can see the dataset has 2260701 rows and 151 columns. Not all of this data will be useful for the intended loan classification analysis.

## Removing Extraneous Data


**Columns with 80% missing values**

These columns contain no information and will not be useful for any analysis. Consequently, these columns are removed from the dataset in the code below.

In [67]:
# Removing columns with 80% missing values
limitPer = len(accepted_loans) * .80
accepted_loans = accepted_loans.dropna(thresh=limitPer, axis=1)

In [68]:
accepted_loans.shape

(2260701, 93)

Following the removal, there are 93 columns left in the dataset, indicating that 58 columns were quite empty.

**Removing columns based on descriptions**

All the remaining columns in the dataset are reviewed based on descriptions found in the [Lending Club Data Dictionary](https://resources.lendingclub.com/LCDataDictionary.xlsx). Columns which provide information an investor will not have at the time he/she is deciding whether to make an investment (leaks information from the future) are removed from the dataset. Additionally, columns which contain information that is not useful for loan classification (e.g url and member id columns) are also removed. 

The names of the 69 columns left in the dataset are shown below:

In [69]:
# displaying names of columns in the dataset
accepted_loans.columns

Index(['id', 'loan_amnt', 'funded_amnt', 'funded_amnt_inv', 'term', 'int_rate', 'installment', 'grade', 'sub_grade', 'emp_title', 'emp_length', 'home_ownership', 'annual_inc', 'verification_status', 'issue_d', 'loan_status', 'pymnt_plan', 'url', 'purpose', 'title', 'zip_code', 'addr_state', 'dti', 'delinq_2yrs', 'earliest_cr_line', 'fico_range_low', 'fico_range_high', 'inq_last_6mths', 'open_acc', 'pub_rec', 'revol_bal', 'revol_util', 'total_acc', 'initial_list_status', 'out_prncp', 'out_prncp_inv', 'total_pymnt', 'total_pymnt_inv', 'total_rec_prncp', 'total_rec_int', 'total_rec_late_fee', 'recoveries', 'collection_recovery_fee', 'last_pymnt_d', 'last_pymnt_amnt', 'last_credit_pull_d', 'last_fico_range_high', 'last_fico_range_low', 'collections_12_mths_ex_med', 'policy_code', 'application_type', 'acc_now_delinq', 'tot_coll_amt', 'tot_cur_bal', 'total_rev_hi_lim', 'acc_open_past_24mths', 'avg_cur_bal', 'bc_open_to_buy', 'bc_util', 'chargeoff_within_12_mths', 'delinq_amnt',
       'mo_si

From the above list, the columns are not needed for the intended analysis are listed in tables below. The first table below shows the columns that are removed because they contain information not needed.

|Count| Column title         | Description                                                                          |
|-----|----------------------|--------------------------------------------------------------------------------------|
|  1  |id                    |A random unique identifier created by Lending Club                                    |
|  2  |url                   |A URL link to the loan                                                                |
|  3  |zip_code              |The first 3 letters of the zipcode, provides the same information as addr_state       |


The table below shows columns removed because they leak information from the future.

|Count| Column title         | Description                                                                          |
|-----|----------------------|--------------------------------------------------------------------------------------|
|  1  |funded_amnt           |Amount of the loan that was funded by investors and Lending Club                      |
|  2  |funded_amnt_inv       |Amount of the loan investors funded                                                   |
|  3  |issue_d               |The month which loan was funded                                                       |
|  4  |out_prncp             |Outstanding principal                                                                 |
|  5  |out_prncp_inv         |Outstanding principal investors portion of fund                                       |
|  6  |total_pymnt| Payments received to date on loan funded|
|  7  |total_pymnt_inv| Payments received to date on loan funded|
|  8  |total_rec_prncp|Principal received to date|
|  9  |total_rec_int|Interest received to date|
| 10  |total_rec_late_fees|Recovered late fees|
| 11  |recoveries| Post charge off gross recoveries|
| 12  |collection_recovery_fee|Post charge off collection fee|
| 13  |last_pymnt_d | The date the last payment was received|
| 14  |last_pymnt_amnt|The most recent payment amount|
| 15  |last_credit_pull_d|The date of last credit pull by LC|
| 16  |next_pymnt_d|provides information on loans funded 
| 17  |last_fico_range_high|highest FICO score in most recent credit pull|
| 18  |last_fico_range_low |lowest FICO score in most recent credit pull|
| 19  |debt_settlement_flag|settlement of debt following inability to pay|
| 20  |debt_settlement_flag_date|date of debt settlement flag|
| 21  |settlement_date|date of settlement|
| 22  |chargeoff_within_12_mths|provides information on early defaulters|
| 23  |settlement_status|status of debt settlement|
| 24  |settlement_amount|amount to be paid for debt settlement|
| 25  |settlement_percentage|settlement amount as percentage of unpaid debt|
| 26  |settlement_term|time over which settlement is to be paid|
| 27  |hardship_flag|indicates borrowers facing difficulties repaying loans|

NOTE: while issue_d is on this list, it will not be dropped immediately.

In [70]:
# creating a list of the columns listed above 
cols_to_drop = ['id', 'funded_amnt', 'funded_amnt_inv', 'url', 'next_pymnt_d',
                'zip_code', 'out_prncp', 'out_prncp_inv', 'total_pymnt', 
                'total_pymnt_inv', 'total_rec_prncp', 'debt_settlement_flag',
                'total_rec_late_fee', 'recoveries', 'collection_recovery_fee',
                'last_pymnt_d', 'last_pymnt_amnt', 'last_credit_pull_d',
                'last_fico_range_high', 'last_fico_range_low', 'total_rec_int',
                'debt_settlement_flag_date', 'settlement_date', 'hardship_flag',
                'settlement_status', 'settlement_amount', 'settlement_term',
                'settlement_percentage', 'chargeoff_within_12_mths', 'total_rev_hi_lim', 'disbursement_method']
 
# dropping the columns listed above  
accepted_loans = accepted_loans.drop(cols_to_drop, axis=1, errors='ignore')

The descriptions alone were not enough to decide whether to drop certain columns. The columns listed below require further investigation to decide how to deal with them:
* fico_range_high and fico_range_low
* purpose and title 
* earliest_cr_line and issue_d

**FICO score columns:** The fico_range_high and fico_range_low columns display represent the region within which a borrower's FICO score is in. There are 44 unique ranges. Having the range values in two columns is inefficient as the average of the range can be used to form one categorical column. 

In [71]:
# creating the fico_average column
accepted_loans['fico_average'] = (accepted_loans['fico_range_high'] + accepted_loans['fico_range_low'])/2

# dropping the fico range columns 
accepted_loans = accepted_loans.drop(['fico_range_low','fico_range_high'], axis=1)

**Purpose and title columns:** The purpose and tilte columns are both provided by the borrower. The purpose column contains categorical information on the purpose of the loan while the title column contains the name the borrower assigns the loan. These two columns contain the very similar information however, the purpose column is better categorized (as shown below). For this reason, the title column is dropped.

In [72]:
# printing the number of unique values in each column
print('Number of unique values in the purpose column: ' + str(accepted_loans['purpose'].nunique()))
print('Number of unique values in the title column: ' + str(accepted_loans['title'].nunique()))

# dropping the title column
accepted_loans = accepted_loans.drop(['title'], axis=1)

Number of unique values in the purpose column: 14
Number of unique values in the title column: 63155


**Earliest Credit line:** An important feature when determining credit scores is the age of the oldest account. For this reason, the earliest_cr_line column will be engineered to estimate the age of each borrowers oldest account. This will be achieved by calculating the difference between the month which a loan was funded (issue_d) and the borrower's earliest credit line (earliest_cr_line). It is a reasonable approximation as it provides a good estimate of the age of credit line an investor will see when deciding whether to invest in the loan.

In [73]:
# converting the earliest credit line column to datetime
accepted_loans['earliest_cr_line']= pd.to_datetime(accepted_loans['earliest_cr_line'])

# converting the loan issue date column to datetime
accepted_loans['issue_d'] = pd.to_datetime(accepted_loans['issue_d'])

# estimating the age of the oldest credit line
accepted_loans['age_cr_line'] = accepted_loans['issue_d']- accepted_loans['earliest_cr_line'] 

# dropping the earliest credit line and loan issue date columns
accepted_loans = accepted_loans.drop(['earliest_cr_line', 'issue_d'], axis =1)

# converting from time delta to numeric type
accepted_loans['age_cr_line'] = accepted_loans['age_cr_line'].dt.days

# converting age of credit line from days to months
days_in_month = 30.4375
accepted_loans['age_cr_line'] = round(accepted_loans['age_cr_line']/days_in_month)

In [74]:
# print the size of the dataset
print('The size of the dataset: ' + str(accepted_loans.shape))

The size of the dataset: (2260701, 66)


**Removing columns with one unique value**

Following the review of each description, there are 65 columns left in the dataset. In this section, columns that have only one unique value are removed. 

In [75]:
# removing columns with only one unique value 
accepted_loans = accepted_loans.loc[:,accepted_loans.apply(func=pd.Series.nunique, args=(False)) > 1]

# printing the size of the dataset
print('The size of the dataset: ' + str(accepted_loans.shape))

The size of the dataset: (2260701, 65)


There are now 65 columns remaining. The name of these columns and their descriptions are shown below.

In [76]:
# importing the Lending Club data dictionary
LC_dictionary = pd.read_excel('Data_Dictionary.xlsx', index_col = 0)

# removing unwanted columns
LC_dictionary = LC_dictionary.iloc[:,:1]

# ensuring descriptions are not truncated
pd.options.display.max_colwidth =300

# adding descriptions for newly created columns
LC_dictionary.loc['fico_average'] = 'The average FICO score of the borrower at the time of application'
LC_dictionary.loc['age_cr_line'] = 'Estimated age of borrowers oldest credit account at the time of the appliation'

# displaying the descriptions of these columns 
LC_dictionary.loc[accepted_loans.columns.tolist(), :]

,Description
LoanStatNew,
loan_amnt,"The listed amount of the loan applied for by the borrower. If at some point in time, the credit department reduces the loan amount, then it will be reflected in this value."
term,The number of payments on the loan. Values are in months and can be either 36 or 60.
int_rate,Interest Rate on the loan
installment,The monthly payment owed by the borrower if the loan originates.
grade,LC assigned loan grade
sub_grade,LC assigned loan subgrade
emp_title,The job title supplied by the Borrower when applying for the loan.*
emp_length,Employment length in years. Possible values are between 0 and 10 where 0 means less than one year and 10 means ten or more years.
home_ownership,"The home ownership status provided by the borrower during registration. Our values are: RENT, OWN, MORTGAGE, OTHER."


In [57]:
# resetting the max column width display 
pd.options.display.max_colwidth = 50
pd.set_option('display.max_rows', 500)
pd.set_option('display.max_columns', 500)
pd.set_option('display.width', 1000)

## Preparing features for data exploration and machine learning

### Wrangling Categorical columns

**Categorizing the employer title column**

As fully categorizing the employer title column will be significant work, only some of the most common or popular employer titles are categorized. The categories used include:
1. No response
2. Unemployed
3. Self employed
4. Educational and Reserch Institutions
5. US Military 
6. Big Financial Services
7. Technology Companies (FAANG)
8. Other employers

In [77]:
# creating a function to categorize employrt title
def employer_categorizer(emp_title):
    
    # creating a placeholder for the category
    category = int()
    
    try:
        # creating a list of unemployed/employed words
        unemploy = ["unemploy", 'retired', 'un-employ', 'un employ']
        
        # check if unemployed
        if any(word in emp_title.lower() for word in unemploy):
            if any('retirement' in emp_title.lower() for word in unemploy):
                category = emp_title
            else:
                category = 'unemployed'
            
        # check if self-employed
        elif "self" in emp_title.lower():
            
            # some key words attached to self that may not be self-employed
            not_self_emp = ['storage', 'housing', 'elderly']
            
            if any(word in emp_title.lower() for word in not_self_emp):
                category = emp_title
             
            else:
                category = 'self_employed'
        
        # checking for other job categories
        if (category != 'self_employed') & (category != 'unemployed'):
            
            # list of words for military related employer title
            US_military = ["usaf", 'army', "air force", "marine corps", "patrol", "navy",
                           "military", "usmc", "coast guard"]
            
            # list of words for big financial services related work (big  investment/accounting/consulting)
            Financial_services = ["bank of america", "jp", "chase", "wells", "morgan", 'deloitte',
                                  "fidelity", "american express", "lynch","hsbc", "barclays",
                                  "capital one", 'kpmg',"schwab", 'pricewater', 'arthur anderson' 
                                  'goldman',"accenture", "bcg", "boston consulting", "ernst",
                                  "bain", 'mc kinsey', 'mckinsey']
            
            # list of words for telecommunications/tech related 
            Big_tech = ['facebook', 'fb', 'google', 'alphabet', 'netflix', 'apple', 'amazon']
            
            # list of words for educational/health related services
            Education = ['college', 'university', 'school', 'education','ucsf', 'institute',
                         'research']
            # big retailers
            Big_retailers =  ['walmart', 'walgreens', 'target', 'cvs', 'best buy',  'safeway',
                              'depot', 'nordstrom', 'costco', 'wal-mart', 'rite aid', 'staples', 
                              'macy\'s', 'macy', 'kroger', 'albertson', 'nordstrom', 'lowe\'s', 
                              'kohl', 'aldi', 'publix']

            
            # check if borrower works in the US military
            if any(employer in emp_title.lower() for employer in US_military):
                category = 'US Military' 
            
            # check if borrower works in financial/consulting services
            elif any(employer in emp_title.lower() for employer in Financial_services):
                category = 'Big Financial Services'
            
            # check if borrower works in telecommunications
            elif any(employer in emp_title.lower() for employer in Big_tech):
                category = 'Tech companies (FAANG)'
            
            # check if employer works in education
            elif any(employer in emp_title.lower() for employer in Education):
                category = 'Educational Institutions'
            
            # check if employer works in education
            elif any(employer in emp_title.lower() for employer in Big_retailers):
                category = 'Major Retailers'
        
            else:
                category = 'Uncategorized employer'
               
    except Exception:
        # check if no response
        if math.isnan(emp_title):
            category = 'No response'
    
    return category

# creating categorical employer title columns
accepted_loans['emp_title_cat'] = accepted_loans.emp_title.apply(employer_categorizer)

# dropping the employer_title column
accepted_loans = accepted_loans.drop(['emp_title'], axis =1)

# viewing the results 
accepted_loans['emp_title_cat'].value_counts()

Uncategorized employer      2044190
No response                  167002
Educational Institutions      28549
Major Retailers                8840
US Military                    6547
Big Financial Services         4276
Tech companies (FAANG)          611
self_employed                   488
unemployed                      198
Name: emp_title_cat, dtype: int64

**Using ordinal values to categorize the employment length and sub grade columns**

The employment length and subgrade columns are converted to numeric type for data exploration and machine learning. For the employment length column, 10 or more years of employment is categorized 10 years of employment, while the "n/a" responses and responses indicating less than 1 year of experience are categorized as 0 years of employment. 

The maps for the employment and sub grade columns are shown below:

In [78]:
# creating a mapping dictionary for the sub_grade column
ranked_sub_grade = accepted_loans.sub_grade.value_counts().sort_index().index.tolist()
sub_grade_map = {sub_grade:(index+1) for index, sub_grade in enumerate(ranked_sub_grade)}

# creating a mapping dictionary for the grade column
ranked_grade = accepted_loans.grade.value_counts().sort_index().index.tolist()
grade_map = {grade:(index+1) for index, grade in enumerate(ranked_grade)}

# Map for the employment length column
mapping_dict = {"emp_length": {
                               "10+ years": 10,
                               "9 years": 9,
                               "8 years": 8,
                               "7 years": 7,
                               "6 years": 6,
                               "5 years": 5,
                               "4 years": 4,
                               "3 years": 3,
                               "2 years": 2,
                               "1 year": 1,
                               "< 1 year": 0,
                               "n/a": 0
                               },
                "sub_grade": sub_grade_map,
                "grade": grade_map
               }
               


# converting the columns
accepted_loans = accepted_loans.replace(mapping_dict)

**Using dummy columns to categorize the nominal variables**

Since nominal variables cannot be ranked, dummy columns will be made to categorize them. This code for this is shown below for the columns: "home_ownership", "verification_status", "purpose", "term" and "emp_title_cat".

In [60]:
# # creating a list of nominal columns
# nominal_columns = ["home_ownership", "verification_status", "purpose", "term", 
#                    "emp_title_cat"]

# # creating dummy columns 
# dummy_df = pd.get_dummies(accepted_loans[nominal_columns], drop_first = True)

# # concatenating the columns to loan_data dataframe
# loan_data = pd.concat([accepted_loans, dummy_df], axis=1)

# # dropping the nominal columns
# loan_data = accepted_loans.drop(nominal_columns, axis=1)

The addr_state column contains too many nominal variables. For this reason, only states with more than 1% of the total borrowers in the dataset will be converted into dummy variables. All other states are categorized based on the region of the country they are in (West, Midwest, Northeast and South).

In [82]:
# making a list of states with more than 420 borrowers  
state_count = accepted_loans['addr_state'].value_counts()
top_counts = state_count[state_count>420]
top_states = top_counts.index.tolist()

# creating a function to categorize states by region
def state_categorizer(states):
    
    # making lists of states and Washington DC  by region
    West = ["CA", "OR", "NV", "WA", "ID", "UT", "AZ", "NM", "CO", "WY", "MT", 
            "AK","HI"]
    Midwest = ["MD", "MN", "WY", "SD", "NE", "KS", "MO", "IA", "WI", "IL", "MI",
               "IN", "OH"]
    Northeast = ["ME", "NH", "VT", 'PA', "CT", "NY", "MA", "CT", "NJ", "RI"]
    South = ["TX", "OK", "AR", "LA", "MS", "AL", "TN", "KY", "GA", "FL", "SC"
             "NC", "VA", "WV", "DC", "MD", "DE"]
      
    try: 
    # check which category state belongs to
        if any(state in states for state in top_states):
            category  = states
        elif any(state in states for state in West):
            category = 'West'
        elif any(state in states for state in Midwest):
            category = 'Midwest'
        elif any(state in states for state in South):
            category = 'South'
        elif any(state in states for state in Northeast):
            category = 'Northeast'
        else:
            category = 'uncategorized'
        
    except Exception:
            category = 'no response'
    
    return category

# creating categorical employer title columns
accepted_loans['categorized_states'] = accepted_loans.addr_state.apply(state_categorizer)
accepted_loans = accepted_loans.drop('addr_state', axis=1)

# # making dummy region columns
# dummy_region = pd.get_dummies(accepted_loans['categorized_states'], drop_first = True)

# # concatenating the columns to loan_data dataframe
# accepted_loans = pd.concat([accepted_loans, dummy_region], axis=1)

# # dropping the region and addr_state columns
# accepted_loans = accepted_loans.drop(['addr_state', 'categorized_states'], axis=1)

### Preparing the target column
The target column for loan classification is the loan_status column. A quick look at the variables and their respective counts in the loan_status column is shown below.

In [84]:
# showing variables and count
accepted_loans['loan_status'].value_counts()

Fully Paid                                             1076751
Current                                                 878317
Charged Off                                             268559
Late (31-120 days)                                       21467
In Grace Period                                           8436
Late (16-30 days)                                         4349
Does not meet the credit policy. Status:Fully Paid        1988
Does not meet the credit policy. Status:Charged Off        761
Default                                                     40
Name: loan_status, dtype: int64

According to Lending Club, loans of the type which does not meet their credit policy will no more be offered to investors. Consequently, these columns will be discarded and the remaining rows will be categorized such that:
* Fully Paid: 1
* Charged Off: 0

In [85]:
# removing rows that do not meet Lending Club's credit policy
target = [1 if i in ['Default', 'Charged Off'] else 0 for i in accepted_loans['loan_status']]
accepted_loans['target'] = target
accepted_loans['target'].value_counts()

0    1992102
1     268599
Name: target, dtype: int64

In [96]:
accepted_loans = accepted_loans.drop('loan_status', axis=1)

In [97]:
accepted_loans.shape

(2260701, 65)

There are currently 116 columns. Reducing the number of rows may have affected the number of unique values in some columns. Once again the columns with only one unique value are removed.

In [21]:
# removing columns with only one unique value 
loan_data = loan_data.loc[:,loan_data.apply(func=pd.Series.nunique, args=(False)) > 1]

# printing the size of the dataset
print('The size of the dataset: ' + str(loan_data.shape))

The size of the dataset: (39786, 79)


### Handling Missing Values
With the categorical columns prepared, missing values will now be handled. Below we take a look at the missing values count in columns with missing values.



In [22]:
# counting the number of missing values
null_counts = loan_data.isnull().sum()

# displaying results 
null_counts[null_counts != 0]

emp_length                 1078
mths_since_last_delinq    25727
mths_since_last_record    36995
revol_util                   50
pub_rec_bankruptcies        697
dtype: int64

**Strategy for handling missing revolving utilization missing values:**
* There are 50 rows with missing data in the revolving utilization column. This represents less than 1% (398 rows) of the rows in the data. Since these rows are few and difficult to predict, they will be dropped.

In [23]:
# Dropping rows with missing values in revol_util
loan_data = loan_data[pd.notnull(loan_data['revol_util'])]

# displaying results for the missing values
null_counts = loan_data.isnull().sum()
null_counts[null_counts != 0]

emp_length                 1075
mths_since_last_delinq    25690
mths_since_last_record    36947
pub_rec_bankruptcies        697
dtype: int64

Verifying that dropping these rows did not affect the number of unique values in a column.

In [24]:
# removing columns with only one unique value 
loan_data = loan_data.loc[:,loan_data.apply(func=pd.Series.nunique, args=(False)) > 1]

# printing the size of the dataset
print('The size of the dataset: ' + str(loan_data.shape))

The size of the dataset: (39736, 79)


In [88]:
# displaying results of missing values
null_counts = accepted_loans.isnull().sum()
null_counts[null_counts != 0]

loan_amnt                         33
term                              33
int_rate                          33
installment                       33
grade                             33
sub_grade                         33
emp_length                    146940
home_ownership                    33
annual_inc                        37
verification_status               33
loan_status                       33
pymnt_plan                        33
purpose                           33
dti                             1744
delinq_2yrs                       62
inq_last_6mths                    63
open_acc                          62
pub_rec                           62
revol_bal                         33
revol_util                      1835
total_acc                         62
initial_list_status               33
collections_12_mths_ex_med       178
application_type                  33
acc_now_delinq                    62
tot_coll_amt                   70309
tot_cur_bal                    70309
a

In [91]:
percent_missing = accepted_loans.isnull().mean() * 100
missing_value_df = pd.DataFrame({'column_name': accepted_loans.columns,
                                 'percent_missing': percent_missing})
missing_value_df.sort_values('percent_missing', inplace=True)
missing_value_df.tail(50)

,column_name,percent_missing
sub_grade,sub_grade,0.001460
term,term,0.001460
annual_inc,annual_inc,0.001637
total_acc,total_acc,0.002743
open_acc,open_acc,0.002743
acc_now_delinq,acc_now_delinq,0.002743
age_cr_line,age_cr_line,0.002743
pub_rec,pub_rec,0.002743
delinq_amnt,delinq_amnt,0.002743
delinq_2yrs,delinq_2yrs,0.002743


The data is now ready for exploration and machine learning. 

In [92]:
# exporting data
accepted_loans.to_csv('accepted_loans_cleaned.csv', index = False)

In [94]:
accepted_loans.columns.to_list()

['loan_amnt',
 'term',
 'int_rate',
 'installment',
 'grade',
 'sub_grade',
 'emp_length',
 'home_ownership',
 'annual_inc',
 'verification_status',
 'loan_status',
 'pymnt_plan',
 'purpose',
 'dti',
 'delinq_2yrs',
 'inq_last_6mths',
 'open_acc',
 'pub_rec',
 'revol_bal',
 'revol_util',
 'total_acc',
 'initial_list_status',
 'collections_12_mths_ex_med',
 'application_type',
 'acc_now_delinq',
 'tot_coll_amt',
 'tot_cur_bal',
 'acc_open_past_24mths',
 'avg_cur_bal',
 'bc_open_to_buy',
 'bc_util',
 'delinq_amnt',
 'mo_sin_old_il_acct',
 'mo_sin_old_rev_tl_op',
 'mo_sin_rcnt_rev_tl_op',
 'mo_sin_rcnt_tl',
 'mort_acc',
 'mths_since_recent_bc',
 'mths_since_recent_inq',
 'num_accts_ever_120_pd',
 'num_actv_bc_tl',
 'num_actv_rev_tl',
 'num_bc_sats',
 'num_bc_tl',
 'num_il_tl',
 'num_op_rev_tl',
 'num_rev_accts',
 'num_rev_tl_bal_gt_0',
 'num_sats',
 'num_tl_120dpd_2m',
 'num_tl_30dpd',
 'num_tl_90g_dpd_24m',
 'num_tl_op_past_12m',
 'pct_tl_nvr_dlq',
 'percent_bc_gt_75',
 'pub_rec_bankrupt